#### Objective:- Create table for generating race migration points.

In [1]:
import pandas as pd
import geopandas as gpd
import ast,os,random
pd.set_option('display.float_format','{:.1f}'.format)
import warnings
warnings.filterwarnings('ignore')
import cudf, cupy as cp
import numpy as np
import time
# pd.set_option('display.max_colwidth', -1)

## Race computation

In [42]:
full_2020_path='data/nhgis0007_csv/nhgis0007_ds248_2020_block.csv'
full_2010_path='data/race2010/nhgis0004_csv/nhgis0004_ds172_2010_block.csv'

In [43]:
race_10 = cudf.read_csv(full_2010_path,usecols=['GISJOIN','STATEA','H7X001','H7X002','H7X003','H7X004','H7X005','H7X006','H7X007','H7X008'])
race_10.head()

,GISJOIN,STATEA,H7X001,H7X002,H7X003,H7X004,H7X005,H7X006,H7X007,H7X008
0,G01000100201001000,01,61,55,4,0,1,0,0,1
1,G01000100201001001,01,0,0,0,0,0,0,0,0
2,G01000100201001002,01,0,0,0,0,0,0,0,0
3,G01000100201001003,01,75,66,4,0,0,0,4,1
4,G01000100201001004,01,0,0,0,0,0,0,0,0


In [44]:
race_10.GISJOIN = race_10.GISJOIN.str.replace('G', '')
race_10.rename(columns={'H7X001':'P10','STATEA':'STATE','H7X002':'R101','H7X003':'R102','H7X004':'R103','H7X005':'R104','H7X006':'R105','H7X007':'R106','H7X008':'R107'},inplace=True)
race_10['ID10'] = (race_10.GISJOIN.str.slice(start=0,stop=2) + race_10.GISJOIN.str.slice(start=3,stop=6) + race_10.GISJOIN.str.slice(start=7)).astype('int64')
race_10.head()

,GISJOIN,STATE,P10,R101,R102,R103,R104,R105,R106,R107,ID10
0,01000100201001000,01,61,55,4,0,1,0,0,1,10010201001000
1,01000100201001001,01,0,0,0,0,0,0,0,0,10010201001001
2,01000100201001002,01,0,0,0,0,0,0,0,0,10010201001002
3,01000100201001003,01,75,66,4,0,0,0,4,1,10010201001003
4,01000100201001004,01,0,0,0,0,0,0,0,0,10010201001004


#### Prepare 2020 data

In [45]:
race_20 = cudf.read_csv(full_2020_path,usecols=['GISJOIN','GEOCODE','STATEA','U7B001','U7B003','U7B004','U7B005','U7B006','U7B007','U7B008','U7B009'],dtype={'U7B001':'float32','U7B003':'float32','U7B004':'float32','U7B005':'float32','U7B006':'float32','U7B007':'float32','U7B008':'float32','U7B009':'float32'})
race_20.head()

,GISJOIN,GEOCODE,STATEA,U7B001,U7B003,U7B004,U7B005,U7B006,U7B007,U7B008,U7B009
0,G01000100201001000,010010201001000,01,21.0,12.0,4.0,0.0,0.0,0.0,0.0,5.0
1,G01000100201001001,010010201001001,01,34.0,18.0,11.0,0.0,2.0,0.0,1.0,2.0
2,G01000100201001002,010010201001002,01,29.0,24.0,2.0,0.0,0.0,0.0,2.0,1.0
3,G01000100201001003,010010201001003,01,17.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0
4,G01000100201001004,010010201001004,01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
# df = small.copy()
race_20.rename(columns={'U7B001':'P20','STATEA':'STATE','GEOCODE':'ID20','U7B003':'R201','U7B004':'R202','U7B005':'R203','U7B006':'R204','U7B007':'R205','U7B008':'R206','U7B009':'R207'},inplace=True)
race_20.GISJOIN = race_20.GISJOIN.str.replace('G', '').astype('int64')
race_20.head()

,GISJOIN,ID20,STATE,P20,R201,R202,R203,R204,R205,R206,R207
0,1000100201001000,010010201001000,01,21.0,12.0,4.0,0.0,0.0,0.0,0.0,5.0
1,1000100201001001,010010201001001,01,34.0,18.0,11.0,0.0,2.0,0.0,1.0,2.0
2,1000100201001002,010010201001002,01,29.0,24.0,2.0,0.0,0.0,0.0,2.0,1.0
3,1000100201001003,010010201001003,01,17.0,16.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1000100201001004,010010201001004,01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Mapper 

#### Concat mapper files for states

In [47]:
states = {1 :"AL",2 :"AK",4 :"AZ",5 :"AR",6 :"CA",8 :"CO",9 :"CT",10:"DE",11:"DC",12:"FL",13:"GA",15:"HI",
          16:"ID",17:"IL",18:"IN",19:"IA",20:"KS",21:"KY",22:"LA",23:"ME",24:"MD",25:"MA",26:"MI",27:"MN",
          28:"MS",29:"MO",30:"MT",31:"NE",32:"NV",33:"NH",34:"NJ",35:"NM",36:"NY",37:"NC",38:"ND",39:"OH",
          40:"OK",41:"OR",42:"PA",44:"RI",45:"SC",46:"SD",47:"TN",48:"TX",49:"UT",50:"VT",51:"VA",53:"WA",
          54:"WV",55:"WI",56:"WY",72:"PR"}

# states = {11:"DC",10:"DE"}


In [48]:
def concat_states_mapper(state_key_list): 
    
    concat_mapper = cudf.DataFrame()
    for i in state_key_list:
        if i< 10:
            i_str = '0'+str(i)
        else:
            i_str = str(i)
        path = 'data/block_rel_files/tab2010_tab2020_st%s_%s.csv'%(i_str,states[i].lower())
        if os.path.isfile(path):    
            mapper = cudf.read_csv(path,delimiter='|')
            concat_mapper = cudf.concat([concat_mapper,mapper])
        else:
            print("mapper file does not exist")
            continue
    return concat_mapper

In [49]:
mapper_df = concat_states_mapper(states.keys())
mapper_df.rename(columns={'﻿STATE_2010':'STATE_2010'},inplace=True)
mapper_df.head(2)

,STATE_2010,COUNTY_2010,TRACT_2010,BLK_2010,BLKSF_2010,AREALAND_2010,AREAWATER_2010,BLOCK_PART_FLAG_O,STATE_2020,COUNTY_2020,TRACT_2020,BLK_2020,BLKSF_2020,AREALAND_2020,AREAWATER_2020,BLOCK_PART_FLAG_R,AREALAND_INT,AREAWATER_INT
0,1,1,20100,1000,<NA>,482628,0,p,1,1,20100,1000,<NA>,288702,0,<NA>,288702,0
1,1,1,20100,1000,<NA>,482628,0,p,1,1,20100,1001,<NA>,194408,0,p,193926,0


In [50]:
mapper_df['ID10'] = (mapper_df.STATE_2010.astype('str').str.rjust(2,'0') + mapper_df.COUNTY_2010.astype('str').str.rjust(3,'0') + mapper_df.TRACT_2010.astype('str').str.rjust(6,'0') + mapper_df.BLK_2010.astype('str').str.rjust(4,'0')).astype('int64')
mapper_df['ID20'] = (mapper_df.STATE_2020.astype('str').str.rjust(2,'0') + mapper_df.COUNTY_2020.astype('str').str.rjust(3,'0') + mapper_df.TRACT_2020.astype('str').str.rjust(6,'0') + mapper_df.BLK_2020.astype('str').str.rjust(4,'0')).astype('int64')
mapper_df = mapper_df[['ID10','ID20','STATE_2020']].reset_index()
mapper_df.head()

,index,ID10,ID20,STATE_2020
0,0,10010201001000,10010201001000,1
1,1,10010201001000,10010201001001,1
2,2,10010201001001,10010201001001,1
3,3,10010201001002,10010201001002,1
4,4,10010201001003,10010201001002,1


#### Create Mapped IDs

In [51]:
freq_df = mapper_df.ID10.value_counts().reset_index().sort_values('index')
freq_df.rename(columns={'ID10':'freq'},inplace=True)
freq_df.rename(columns={'index':'ID10'},inplace=True)
freq_df = freq_df.reset_index(drop=True)
race_10.P10 = race_10.P10.astype('float32')
freq_df = cudf.merge(freq_df,race_10[['ID10','P10','R101','R102','R103','R104','R105','R106','R107']],on='ID10',how='right').sort_values('ID10')
freq_df['P10_weights'] = freq_df['P10'].divide(freq_df['freq'])
freq_df['R101_weights'] = freq_df['R101'].divide(freq_df['freq'])
freq_df['R102_weights'] = freq_df['R102'].divide(freq_df['freq'])
freq_df['R103_weights'] = freq_df['R103'].divide(freq_df['freq'])
freq_df['R104_weights'] = freq_df['R104'].divide(freq_df['freq'])
freq_df['R105_weights'] = freq_df['R105'].divide(freq_df['freq'])
freq_df['R106_weights'] = freq_df['R106'].divide(freq_df['freq'])
freq_df['R107_weights'] = freq_df['R107'].divide(freq_df['freq'])
freq_df.head()

,ID10,P10,R101,R102,R103,R104,R105,R106,R107,freq,P10_weights,R101_weights,R102_weights,R103_weights,R104_weights,R105_weights,R106_weights,R107_weights
2832,10010201001000,61.0,55,4,0,1,0,0,1,2,30.5,27.5,2.0,0.0,0.5,0.0,0.0,0.5
2833,10010201001001,0.0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2834,10010201001002,0.0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2835,10010201001003,75.0,66,4,0,0,0,4,1,2,37.5,33.0,2.0,0.0,0.0,0.0,2.0,0.5
2836,10010201001004,0.0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
weighted_mapper = cudf.merge(mapper_df,freq_df[['ID10','P10_weights','R101_weights','R102_weights','R103_weights','R104_weights','R105_weights','R106_weights','R107_weights']],on='ID10',how='left').sort_values('ID20').reset_index(drop=True)

In [53]:
eq_10 = weighted_mapper.groupby('ID20')[['ID10','P10_weights','R101_weights','R102_weights','R103_weights','R104_weights','R105_weights','R106_weights','R107_weights']].sum().reset_index().sort_values('ID20').reset_index(drop=True) 
del(weighted_mapper)
eq_10.rename(columns={'P10_weights':'eq_P10','R101_weights':'eq_R101','R102_weights':'eq_R102','R103_weights':'eq_R103','R104_weights':'eq_R104','R105_weights':'eq_R105','R106_weights':'eq_R106','R107_weights':'eq_R107'},inplace=True)
eq_10.head()

,ID20,ID10,eq_P10,eq_R101,eq_R102,eq_R103,eq_R104,eq_R105,eq_R106,eq_R107
0,10010201001000,10010201001000,30.5,27.5,2.0,0.0,0.5,0.0,0.0,0.5
1,10010201001001,20020402002001,30.5,27.5,2.0,0.0,0.5,0.0,0.0,0.5
2,10010201001002,50051005005035,51.8,45.3,4.0,0.0,0.0,0.0,2.0,0.5
3,10010201001003,40040804004046,13.3,11.3,2.0,0.0,0.0,0.0,0.0,0.0
4,10010201001004,10010201001008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
race_20.head(2)

,GISJOIN,ID20,STATE,P20,R201,R202,R203,R204,R205,R206,R207
0,1000100201001000,010010201001000,01,21.0,12.0,4.0,0.0,0.0,0.0,0.0,5.0
1,1000100201001001,010010201001001,01,34.0,18.0,11.0,0.0,2.0,0.0,1.0,2.0


In [55]:
print(len(eq_10))
print(len(race_20))

8174955
8174955


#### Compute net population

In [56]:
race_df = eq_10[['ID20']].copy()

In [57]:
race_df['P_diff'] = race_20['P20'].sub(eq_10['eq_P10'])
race_df['R1_diff'] = race_20['R201'].sub(eq_10['eq_R101'])
race_df['R2_diff'] = race_20['R202'].sub(eq_10['eq_R102'])
race_df['R3_diff'] = race_20['R203'].sub(eq_10['eq_R103'])
race_df['R4_diff'] = race_20['R204'].sub(eq_10['eq_R104'])
race_df['R5_diff'] = race_20['R205'].sub(eq_10['eq_R105'])
race_df['R6_diff'] = race_20['R206'].sub(eq_10['eq_R106'])
race_df['R7_diff'] = race_20['R207'].sub(eq_10['eq_R107'])
race_df.head()

,ID20,P_diff,R1_diff,R2_diff,R3_diff,R4_diff,R5_diff,R6_diff,R7_diff
0,10010201001000,-9.5,-15.5,2.0,0.0,-0.5,0.0,0.0,4.5
1,10010201001001,3.5,-9.5,9.0,0.0,1.5,0.0,1.0,1.5
2,10010201001002,-22.8,-21.3,-2.0,0.0,0.0,0.0,0.0,0.5
3,10010201001003,3.7,4.7,-2.0,0.0,0.0,0.0,0.0,1.0
4,10010201001004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
df = cudf.read_csv('data/mapped_blocks_full.csv',dtype={'GISJOIN':'int64','ID20':'int64','STATE':'int32','COUNTY':'str','P20':'int32','P10_new':'int32'}).drop('Unnamed: 0',axis=1)
df['P_delta']=df['P20'] - df['eq_P10']
# df = df[df.number!=0] 
df.head()

,GISJOIN,ID20,STATE,COUNTY,P20,eq_P10,P_delta
0,1000100201001000,10010201001000,1,Autauga County,21,30.5,-9.5
1,1000100201001001,10010201001001,1,Autauga County,34,30.5,3.5
2,1000100201001002,10010201001002,1,Autauga County,29,51.8,-22.8
3,1000100201001003,10010201001003,1,Autauga County,17,13.3,3.7
4,1000100201001004,10010201001004,1,Autauga County,0,0.0,0.0


In [59]:
print(len(race_df))
print(len(df))

8174955
8174955


In [60]:
final_df = cudf.merge(df[['ID20','STATE','COUNTY','P_delta']],race_df.drop(columns=['P_diff']),on='ID20').sort_values('ID20')
del(df,race_10,race_20,race_df,eq_10,freq_df)
final_df.head()

,ID20,STATE,COUNTY,P_delta,R1_diff,R2_diff,R3_diff,R4_diff,R5_diff,R6_diff,R7_diff
3392,10010201001000,1,Autauga County,-9.5,-15.5,2.0,0.0,-0.5,0.0,0.0,4.5
3393,10010201001001,1,Autauga County,3.5,-9.5,9.0,0.0,1.5,0.0,1.0,1.5
3394,10010201001002,1,Autauga County,-22.8,-21.3,-2.0,0.0,0.0,0.0,0.0,0.5
3395,10010201001003,1,Autauga County,3.7,4.7,-2.0,0.0,0.0,0.0,0.0,1.0
3396,10010201001004,1,Autauga County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
final_df['P_delta'] = final_df.P_delta.round().astype('int32')
final_df['number'] = final_df.P_delta.round().abs().astype('int32')
final_df['R1_diff'] = final_df.R1_diff.round().astype('int32')
final_df['R2_diff'] = final_df.R2_diff.round().astype('int32')
final_df['R3_diff'] = final_df.R3_diff.round().astype('int32')
final_df['R4_diff'] = final_df.R4_diff.round().astype('int32')
final_df['R5_diff'] = final_df.R5_diff.round().astype('int32')
final_df['R6_diff'] = final_df.R6_diff.round().astype('int32')
final_df['R7_diff'] = final_df.R7_diff.round().astype('int32')
final_df.head()

,ID20,STATE,COUNTY,P_delta,R1_diff,R2_diff,R3_diff,R4_diff,R5_diff,R6_diff,R7_diff,number
3392,10010201001000,1,Autauga County,-10,-16,2,0,0,0,0,4,10
3393,10010201001001,1,Autauga County,4,-10,9,0,2,0,1,2,4
3394,10010201001002,1,Autauga County,-23,-21,-2,0,0,0,0,0,23
3395,10010201001003,1,Autauga County,4,5,-2,0,0,0,0,1,4
3396,10010201001004,1,Autauga County,0,0,0,0,0,0,0,0,0


In [23]:
final_df.number.sum()

115306867

In [23]:
print(len(final_df))

8174955


In [62]:
final_df['error'] =  final_df['P_delta'] - final_df[['R1_diff','R2_diff','R3_diff','R4_diff','R5_diff','R6_diff','R7_diff']].sum(axis=1)
final_df['R7_diff'] =  final_df['R7_diff'] + final_df['error'] # fix error by adjusting R7
print(len(final_df[final_df['P_delta']!= final_df[['R1_diff','R2_diff','R3_diff','R4_diff','R5_diff','R6_diff','R7_diff']].sum(axis=1)]))

0


In [63]:
final_df.iloc[:,4:-2].abs().sum().sum()

182532663

In [64]:
final_df.drop(columns=['error'],inplace=True)
final_df.head(3)

,ID20,STATE,COUNTY,P_delta,R1_diff,R2_diff,R3_diff,R4_diff,R5_diff,R6_diff,R7_diff,number
3392,10010201001000,1,Autauga County,-10,-16,2,0,0,0,0,4,10
3393,10010201001001,1,Autauga County,4,-10,9,0,2,0,1,2,4
3394,10010201001002,1,Autauga County,-23,-21,-2,0,0,0,0,0,23


In [65]:
final_df.to_csv('data/race_migration_diff.csv')

In [66]:
races =  cudf.read_csv('data/race_migration_diff.csv')
races.head()

,Unnamed: 0,ID20,STATE,COUNTY,P_delta,R1_diff,R2_diff,R3_diff,R4_diff,R5_diff,R6_diff,R7_diff,number
0,3392,10010201001000,1,Autauga County,-10,-16,2,0,0,0,0,4,10
1,3393,10010201001001,1,Autauga County,4,-10,9,0,2,0,1,2,4
2,3394,10010201001002,1,Autauga County,-23,-21,-2,0,0,0,0,0,23
3,3395,10010201001003,1,Autauga County,4,5,-2,0,0,0,0,1,4
4,3396,10010201001004,1,Autauga County,0,0,0,0,0,0,0,0,0


In [67]:
races['points']= races[['R1_diff','R2_diff','R3_diff','R4_diff','R5_diff','R6_diff','R7_diff']].abs().sum(axis=1)

In [68]:
len(races)

8174955

In [69]:
races = races[races['points']!=0]
print(len(races))
races.head()

6194258


,Unnamed: 0,ID20,STATE,COUNTY,P_delta,R1_diff,R2_diff,R3_diff,R4_diff,R5_diff,R6_diff,R7_diff,number,points
0,3392,10010201001000,1,Autauga County,-10,-16,2,0,0,0,0,4,10,22
1,3393,10010201001001,1,Autauga County,4,-10,9,0,2,0,1,2,4,24
2,3394,10010201001002,1,Autauga County,-23,-21,-2,0,0,0,0,0,23,23
3,3395,10010201001003,1,Autauga County,4,5,-2,0,0,0,0,1,4,8
5,3397,10010201001005,1,Autauga County,-8,-7,0,0,0,0,0,-1,8,8


In [70]:
races.drop(columns=['Unnamed: 0'],inplace=True)

In [71]:
races.head()

,ID20,STATE,COUNTY,P_delta,R1_diff,R2_diff,R3_diff,R4_diff,R5_diff,R6_diff,R7_diff,number,points
0,10010201001000,1,Autauga County,-10,-16,2,0,0,0,0,4,10,22
1,10010201001001,1,Autauga County,4,-10,9,0,2,0,1,2,4,24
2,10010201001002,1,Autauga County,-23,-21,-2,0,0,0,0,0,23,23
3,10010201001003,1,Autauga County,4,5,-2,0,0,0,0,1,4,8
5,10010201001005,1,Autauga County,-8,-7,0,0,0,0,0,-1,8,8


In [72]:
races.to_csv('data/mapped_data_with_race.csv')